In [ ]:
import math
import random
import numpy as np


class loader:
    def __init__(self,
                 dimentions = 2,
                 trainPercent = 85.0):
        self.__train_procent = trainPercent
        self.__train, self.__test = self.__loadData(dimentions)


    def __loadData(self, dim):
        data = self.__get2DData() if dim == 2 else self.__get3DData()
        ln = len(data)
        lnts = int(ln * (1 - self.__train_procent / 100))
        lntr = ln - lnts

        random.shuffle(data)
        return sorted(data[:lntr]), sorted(data[lntr:])


    def __get2DData(self):
        return [
            [
                [i / 10],
                [math.cos(i / 10) + random.random() * 0.2 - 0.1]
                #[((-i / 10)**3 / 3 + 6*(i/10) - 5) + random.random() * 0.2 - 0.1]
            ]
            for i in range(-60, 61)
        ]


    def __get3DData(self):
        return [[i]]


    def getTrainInp(self):
        return np.array([i[0] for i in self.__train])


    def getTrainOut(self):
        return np.array([i[1] for i in self.__train])


    def getTestInp(self):
        return np.array([i[0] for i in self.__test])

    def getTestOut(self):
        return np.array([i[1] for i in self.__test])

In [ ]:

class MLP:
    __a_parameter_act_functionct_function = 1
    __b_parameter_act_function = 1

    def __init__(self,
                 ld: loader,
                 neuronNum: tuple = (2)
                 ):
        # Количество слоев
        self.__layers = 2 + len(neuronNum)
        # Количество нейронов на каждом слое
        input = ld.getTrainInp()
        out = ld.getTrainOut()
        nN = [len(input[0]), len(out[0])]
        self.__nN = np.insert(nN, 1, neuronNum)
        self.__input = np.array(input)
        self.__output = np.array(out)
        self.__test_input = np.array(ld.getTestInp())
        self.__test_output = np.array(ld.getTestOut())
        self.__w = [np.random.rand(
            self.__nN[i] + 1,
            self.__nN[i + 1] +
            (0 if i == self.__layers - 2 else 1)
        ) for i in range(self.__layers - 1)]


    def non_Linear_activation(self, x):
        return np.array(self.__a_parameter_act_function * np.tanh(self.__b_parameter_act_function * x))


    def non_Linear_activationDer(self, x):
        return np.array(self.__b_parameter_act_function / self.__a_parameter_act_function * \
                        (self.__a_parameter_act_function - self.non_Linear_activation(x)) * \
                        (self.__a_parameter_act_function + self.non_Linear_activation(x)))


    def LinearActivationF(self, x):
        return np.array(x)


    def LinearActivationFDer(self, x):
        return np.array(1)

    def fit(self,
              eta = 0.005,
              epoches=1000,
              epsilon=0.0001):
        full_error_train = []
        full_error_test = []
        # Индуцированное локальное поле или просто сумматор
        v = np.array([None for i in range(self.__layers)])
        # Слои (выходы из слоёв)
        l = np.array([None for i in range(self.__layers)])
        # Ошибки
        l_err = np.array([None for i in range(1, self.__layers)])
        # deltas
        l_delta = np.array([None for i in range(1, self.__layers)])

        # Переобозначение входов и выходов для простоты записи
        input = self.__input
        output = self.__output
        # Счетчик эпох
        count_epoch = 0
        # Начало процесса обучения
        while count_epoch < 2 or count_epoch < epoches and (abs(full_error_test[count_epoch-1] - full_error_test[count_epoch-2]) > epsilon**(3/2) or full_error_test[count_epoch-1] > epsilon):
            count_epoch += 1
            # Проход по обучающей выборке
            for i in range(len(input)):
                l[0] = np.array([np.insert(input[i], 0, 1)])
                # Прямой проход по сети
                for j in range(1, self.__layers - 1):
                    # индуцированное локальное поле
                    v[j] = np.dot(l[j - 1], self.__w[j - 1])
                    # проход по скрытым слоям
                    l[j] = self.non_Linear_activation(v[j])
                # вычисление индуцированного локального поля на выходном слое
                v[self.__layers - 1] = np.dot(l[self.__layers - 2], self.__w[self.__layers - 2])
                # вычисление значения на выходном слое
                l[self.__layers - 1] = self.LinearActivationF(
                    v[self.__layers - 1]
                )
                # Обратный проход по сети
                l_err[self.__layers - 2] = output[i] - l[self.__layers - 1]
                # Нахождение \delta_k
                l_delta[self.__layers - 2] = \
                    np.array([l_err[self.__layers - 2][0] * (
                        self.LinearActivationFDer(v[self.__layers - 1])
                    )])
                # Нахождение \delta_j
                for j in range(self.__layers - 2, 0, -1):
                    l_err[j - 1] = np.dot(l_delta[j], self.__w[j].T)
                    l_delta[j - 1] = l_err[j - 1] * self.non_Linear_activationDer(v[j])
                # Определение изменения весовых коэффициентов \Delta w
                deltaW = [eta * np.dot(l_delta[j].T, l[j])
                          for j in range(self.__layers - 1)]
                for j in range(0, self.__layers - 1):
                    self.__w[j] += deltaW[j].T
            #Ошибка на тестовом множестве
            outputs = self.predict(self.__test_input)
            r_outputs = np.array([self.__test_output[i][0] for i in range(len(self.__test_output))])
            err_n = np.sum(0.5 * (r_outputs - outputs) ** 2) / len(outputs)
            full_error_test.append(err_n)
            #Ошибка на обучающем множестве
            outputr = self.predict(self.__input)
            r_outputr = np.array([self.__output[i][0] for i in range(len(self.__output))])
            tr_err_n = np.sum(0.5 * (r_outputr - outputr) ** 2) / len(outputr)
            full_error_train.append(tr_err_n)
            print("Epoche", count_epoch, "Train error=", tr_err_n, "Test error=", err_n)
        return full_error_train, full_error_test

    #Вычисление выходов по входам обученной сетью
    def predict(self, inputs):
        outs = np.array([])
        # Для каждого входного значения
        for i in range(len(inputs)):
            input = np.array([np.insert(inputs[i], 0, 1)])
            # Прямой проход по сети (все слои, кроме последнего)
            for lr in range(self.__layers - 2):
                input = self.non_Linear_activation(np.dot(input, self.__w[lr]))
            # Получение результата на последнем слое
            # и добавлени его в массив выходов
            outs = np.append(outs,
                             self.LinearActivationF(np.dot(input, self.__w[self.__layers-2]))
                            )
        return outs

In [ ]:

from matplotlib import pyplot as plt

def plot2D():
    ld = loader(trainPercent=85)
    tri = getTrainInp()
    tro = getTrainOut()
    tsi = getTestInp()
    tso = getTestOut()

    mlp = MLP(ld, (12, 30))
    e_tr, e_ts = mlp.fit(epsilon= 25e-4)
    e_ts_x = [i for i in range(1, len(e_ts) + 1)]
    f1 = plt.figure(1)
    fa1 = f1.add_subplot(1, 1, 1)
    out = mlp.predict(tri)
    fa1.plot(tri, out, "bo")
    fa1.plot(tri, tro, "r+")

    out = mlp.predict(tsi)
    fa1.plot(tsi, out, "gv")
    fa1.plot(tsi, tso, "y+")

    f2 = plt.figure(2)
    fa2 = f2.add_subplot(1, 1, 1)
    fa2.plot(e_ts_x, e_ts, "b-")
    plt.show()


plot2D()

NameError: name 'getTrainInp' is not defined

In [ ]:
x = np.linspace(0,1,50)
y = np.linspace(0,1,50)
z = np.cos(x) + np.sin(y)

In [ ]:
nN  = (2,2,2,2,2)

In [ ]:
[np.random.rand( nN[i] + 1, nN[i + 1] + (0 if i == 7 - 2 else 1) ) for i in range(4)]


[array([[0.25729292, 0.78352447, 0.25475914],
        [0.87502656, 0.28784482, 0.09202713],
        [0.37740197, 0.13506615, 0.88208798]]),
 array([[0.9701454 , 0.22870069, 0.7283831 ],
        [0.26224461, 0.19690562, 0.33858703],
        [0.37389501, 0.71063889, 0.46044928]]),
 array([[0.58539984, 0.36791218, 0.0236381 ],
        [0.59236371, 0.11288546, 0.22082592],
        [0.59885126, 0.2383817 , 0.05721669]]),
 array([[0.34761408, 0.31810777, 0.92414625],
        [0.97902784, 0.97475199, 0.79004048],
        [0.29875395, 0.99434187, 0.6830058 ]])]

In [ ]:
neuronNum = (5)
nN = np.insert(nN, 1, neuronNum)


In [ ]:
nN

array([2, 5, 2, 2, 2, 2, 2])